<a href="https://colab.research.google.com/github/MarianaGCosta/Processamento-de-dados-em-portugues-brasileiro/blob/main/Treinamento_spaCy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importação das bibliotes iniciais

In [ ]:
import numpy as np
import pandas as pd
import os
import spacy

from datetime import datetime
import matplotlib.pyplot as plt

## Configuração de diretório de trabalho

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

os.chdir("/content/drive/MyDrive/Corpus_DeG/treinamento_spaCy")

print(os.getcwd())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Corpus_DeG/treinamento_spaCy


## Treinamento spaCy

## Carregamento de dados

In [ ]:
# Carregamento de arquivo lexicográfico

portilexicon = pd.read_csv('portilexicon-ud - portilexicon-ud.csv')
portilexicon

,a,a.1,ADP,_
0,a,a,NOUN,Gender=Masc|Number=Sing
1,a,o,DET,Definite=Def|Gender=Fem|Number=Sing|PronType=Art
2,a,o,PRON,Case=Acc|Gender=Fem|Number=Sing|Person=3|PronT...
3,a,o,PRON,Gender=Fem|Number=Sing|Person=3|PronType=Dem
4,aba,aba,NOUN,Gender=Fem|Number=Sing
...,...,...,...,...
1226333,úteros,útero,NOUN,Gender=Masc|Number=Plur
1226334,útil,útil,ADJ,Number=Sing
1226335,útil,útil,NOUN,Number=Sing
1226336,úvula,úvula,NOUN,Gender=Fem|Number=Sing


In [ ]:
# Renomear as colunas relativas as palavras e aos lemas
portilexicon = portilexicon.rename(columns={"a": "palavras", "a.1": "lemmas"})

In [ ]:
# Transcrever os valores das colunas de palavras e lemas como dados de treino em dicionario

train = {
    "text": portilexicon["palavras"].values.tolist(),
    "annotations": {"label": portilexicon["lemmas"].values.tolist()
                    }
}

In [ ]:
# Exportar o dicionário em um arquivo json

import json
with open('train.json', 'w') as f:
    json.dump(train, f)

In [ ]:
# Verificar se a instalação do spacy foi feita da maneira correta
!pip install -U spacy

In [ ]:
# Importar os pacotes necessários

from spacy.language import Language
from spacy.tokens import Doc
from spacy.tokens import DocBin

In [ ]:
@Language.component("custom_lemma_component")
def custom_lemma_component(doc):

    # Iterar sobre os tokens e substituir pelo lema se encontrado no dicionário
    lemma_dict = dict(zip(portilexicon["palavras"], portilexicon["lemmas"]))

    for token in doc:
        # Verifica se o token está no dicionário e substitui o lema
        if token.text in lemma_dict:
            token.lemma_ = lemma_dict[token.text]
    return lemma

In [ ]:
def get_lemmas(text):
    doc = nlp(text)
    return [token.lemma_ if token.lemma_ != "" else token.text for token in doc]

In [ ]:
# Criar um modelo vazio (sem modelo pré-treinado)
nlp = spacy.blank("pt")

if "lemmatizer" in nlp.pipe_names:
    nlp.remove_pipe("lemmatizer")
    print("Removed lemmatizer")

# Adicionar o componente personalizado ao pipeline
lemmatizer = nlp.add_pipe("custom_lemma_component", last=True)

In [ ]:
# Arquivo corpus em json é uma amostra do dataset do DeG

with open("corpus.json", 'r', encoding='utf-8') as json_file:
    riogrande = json.load(json_file)

In [ ]:
riogrande

['informantes',
 'do',
 'ensino',
 'superior',
 'informante',
 '1',
 'joão',
 'carlos',
 'sexo',
 'masculino',
 'idade',
 '28',
 'anos',
 'data',
 'da',
 'coleta',
 'oral',
 '20',
 '11',
 '93',
 'escrita',
 '21',
 '11',
 '93',
 'e',
 '22',
 '11',
 '93',
 'parte',
 'oral',
 'narrativa',
 'de',
 'experiência',
 'pessoal',
 'e',
 'joão',
 'tens',
 'alguma',
 'história',
 'que',
 'tenha',
 'acontecido',
 'contigo',
 'que',
 'tu',
 'queiras',
 'me',
 'contar',
 'i',
 'tem',
 'uma',
 'recente',
 'eh',
 'pegar',
 'a',
 'lancha',
 'aí',
 'ia',
 'eu',
 'e',
 'dois',
 'amigos',
 'meus',
 'que',
 'trabalham',
 'comigo',
 'lá',
 'o',
 'edson',
 'e',
 'o',
 'miguel',
 'aí',
 'entrei',
 'na',
 'lancha',
 'né',
 'tem',
 'não',
 'sei',
 'se',
 'você',
 'sabe',
 'como',
 'é',
 'que',
 'a',
 'lancha',
 'tem',
 'quatro',
 'uma',
 'fileira',
 'de',
 'quatro',
 'cadeiras',
 'assim',
 'de',
 'madeira',
 'com',
 'encosto',
 'e',
 'a',
 'cadeira',
 'é',
 'normal',
 'aí',
 'até',
 'fui',
 'entrando',
 'convers

## Criação da lematização modelo a partir do recorte do corpus

In [ ]:
recorte_riogrande = [riogrande[:1000]]
recorte_riogrande = recorte_riogrande[0]

In [ ]:
lematizacaomodelo = ['informante', 'de', 'ensino', 'superior', 'informante', '1', 'joão', 'carlos', 'sexo', 'masculino', 'idade', '28', 'ano', 'data', 'de', 'coleta', 'oral', '20', '11', '93', 'escrita', '21', '11', '93', 'e',
'22', '11', '93', 'parte', 'oral', 'narrativa', 'de', 'experiência', 'pessoal', 'e', 'joão', 'ter', 'algum', 'história', 'que', 'ter', 'acontecer', 'contigo', 'que', 'tu', 'querer', 'eu', 'contar', 'i',
'ter', 'um', 'recente', 'eh', 'pegar', 'o', 'lancha', 'aí', 'ir', 'eu', 'e', 'dois', 'amigo', 'meu', 'que', 'trabalhar', 'comigo', 'lá', 'o', 'edson', 'e', 'o', 'miguel', 'aí',
'entrar', 'em', 'lancha', 'né', 'ter', 'não', 'saber', 'se', 'você', 'saber', 'como', 'ser', 'que', 'o', 'lancha', 'ter', 'quatro', 'um', 'fileira', 'de','quatro', 'cadeira','assim',
'de', 'madeira', 'com', 'encosto', 'e', 'o', 'cadeira', 'ser', 'normal', 'aí', 'até', 'ir', 'entrar', 'conversar', 'aí','pedir', 'pra', 'dar', 'lugar', 'pra', 'outro', 'sentar', 'também', 'sentar', 'em', 'canto', 'o', 'outro', 'dois', 'sentar',
'perto','de', 'eu', 'só', 'que', 'eu', 'não', 'ver', 'que', 'estar', 'sem', 'encosto', 'o', 'cadeira', 'eu', 'sentar', 'o','cadeira', 'ser', 'de', 'ser', 'imóvel', 'vir', 'pra', 'trás', 'cair', 'deitado', 'em', 'colo', 'de',
'negrão', 'atrás', 'de', 'eu', 'riso', 'aquilo', 'ali', 'ficar', 'o', 'vale', 'de', 'ano', 'lá', 'em', 'em', 'narrativa', 'recontado', 'e', 'estar', 'joão',  'e', 'ter', 'algum', 'história', 'que', 'alguém',
'ter', 'te', 'contar', 'e', 'que', 'tu', 'não', 'ter', 'participado', 'e', 'que', 'tu', 'saber', 'através', 'de', 'alguém', 'i', 'eh', 'eu', 'não', 'saber', 'se', 'chegar', 'a', 'ser', 'o', 'que', 'tu', 'estar', 'querer',
'mas', 'isso', 'acontecer', 'sexta', 'feira', 'também', 'ser', 'mais', 'pra',  'mais', 'um', 'vale', 'datar', 'o', 'cara', 'chegar', 'estar', 'eh', 'almoçar', 'o', 'mãe', 'dele', 'ligar', 'pra', 'ele', 'assim', '\x93está',
'com', 'um', 'pé', 'de', 'cada','sapato','aí','o','o','colega','chegar','e','disse','\x93ué','pé','de','cada','sapato','você','achar','que','eu','ser','louco','por','um','pé','de','sapato','um','pé','de',
'cada','sapato','olhar','pra','baixo','um','sapato','com','um','lista','e','outro','sem','lista','riso','de','e','ir','trabalhar','com','um','pé','de','cada','sapato','pegar','em','pé','dele','lá','descrição',
'de','local','e','tá','joão','eh','ter','algum','lug','um','lugar','ou','algum','coisa','que','tu','querer','eu','descrever','i','eu','poder','você','descrever','o','agência','lá','em','em','norte','onde',
'eu','trabalhar','eh','logo','em','seguida','qur','tu','descer','do','lancha','ter','o','o','rua','de','frente','ali','álvaro','costa','o','agência','ser','bem','em','frente','ser','dois','piso','né','o',
'setor','de','baixo','ser','onde','funcionar','o','o','caixa','de','gerência','e','em','cima','o','pessoal','mais','o','pessoal','de','de','aplicação','ser','um','um','agência','pequeno','mas','eh','bem',
'bonito','até','relato','de','procedimento','e','tá','e','ter','algum','coisa','que','tu','querer','eu','ensinar','a','fazer','explicar','explicar','como','faz','algum','coisa','pode','ser','algum','coisa',
'relacionado','com','o','você','serviço','algum','i','ser','ser','difícil','ir','dizer','você','ensinar','a','trabalhar','em','caixa','o','o','rapaz','que','fazer','compensação','por','exemplo','quando',
'você','chegar','com','um','depósito','com','cheque','de','outro','banco','tu','tu','tem','que','carimbar','o','cheque','né','aí','em','final','de','dia','tu','tu','somar','ele','banco','por','banco','que',
'ficar','só','pra','pra','compensação','eh','ser','coisa','entender','que','eu','poder','até','você','explicar','mas','não','relato','de','opinião','e','aí','tu','querer','dar','um','opinião','sobre',
'algum','coisa','algum','assunto','i','eh','como','eu','trabalhar','em','banco','agora','que','falar','tudo','agora','sobre','o','sobre','banco','ir','você','dar','um','motivo','um','opinião','sobre',
'não','sei','aplicação','eu','achar','que','pelo','menos','ser','de','se','esperar','como','o','taxa','estar','muito','alta','em','aplicação','dever','ter','um','mudança','em','plano','econômico','de',
'de','país','pro','final','de','ano','até','estar','falar','lá','em','algum','pacote','algum','coisa','porque','a','taxa','estar','muito','al','muito','alta','muito','acima','de','inflação','o','inflação',
'que','ele','divulgar','né','então','por','a','taxa','estar','tão','alta','certamente','dever','ter','algum','mudança','em','em','plano','econômico','e','tá','i','bom','dia','e','obrigado','parte','escrito',
'narrativa','de','experiência','pessoal','ir','descrever','um','acontecimento','engraçado','que','se','passar','comigo','em','lancha','para','são','josé','do','norte','quando','ia','para','o','serviço',
'a','lancha','noivo','de','mar','ter','fileira','com','4','banco','de','madeira','e','o','assento','ser','móvel','entrar','em','lancha','juntamente','com','dois','colega','de','serviço','e','sentei','em',
'cadeira','de','canto','porém','não','ver','que','o','cadeira','estar','sem','encosto','cair','com','tudo','pra','trás','em','colo','de','um','senhor','que','estar','no','banco','de','trás','ser','o',
'vale','do','ano','narrativa','recontado','outro','acontecimento','engraçado','acontecer','outro','dia','com','um','colega','em','serviço','o','mãe','dele','ligar','pra','ele','perguntar','se','ele','estar',
'com','um','pé','de','cada','sapato','ele','ainda','ficar','bravo','dizer','que','não','estar','louco','e','olhar','para','o','pé', 'estar','mesmo','com','um','pé','de','cada','imaginar','o','situação',
'descrição','de','local','ir','descrever','o','agência','de','caixa','econômico','federal','em','são','josé','do','norte','ser','logo','em','rua','em','frente','o','hidroviário','um','agência','de','dois',
'andar','onde','em','parte','inferior','ficar','localizado','o','gerência','o','abertura','de','conta','e','o','caixa','em','andar','superior','ficar','o','cozinha','e','o','banheiro','o','almoxarifado',
'e','o','setor','de','contabilidade','e','aplicação','ser','um','agência','pequeno','porém','bem','agradável','relato','de','procedimento','ir','dar','um','pequeno','procedimento','sobre','o','tratamento',
'dado','a','cheque','de','outro','banco','recebido','pelo','caixa','em','depósito','o','caixa','recebe','o','depósito','em','cheque','carimbar','o','cheque','com','o','carimbo','de','compensação','em',
'verso','e','com','o','de','cruzamento','em','frente','verificar','o','prazo','de','comp','bloqueio','e','autenticar','o','depósito','em','final','de','expediente','fazer','o','soma','de','cheque','verificar',
'se','coincidir','com','o','total','de','depósito','recebido','e','encaminhar','para','o', 'retaguarda','relato', 'de', 'opinião', 'ir', 'dar','um', 'opinião','sobre','o','situação', 'econômico', 'que',
'provavelmente', 'dever', 'ter', 'mudança', 'em', 'final', 'desse', 'ano', 'visto', 'que', 'o', 'taxa', 'de', 'juro','em', 'aplicação', 'em', 'cdb', 'rdb', 'poupança', 'e','outro','estar','muito','elevado',
'em','relação','o', 'inflação', 'divulgado', 'achar', 'que', 'certamente', 'esse', 'equilíbrio', 'provocar', 'mudança', 'e', 'quem', 'saber', 'um', 'novo','pacote', 'econômico', 'informante', '2',
'lisandra','sexo','feminino','idade', '23','ano','data','de','coleta','oral','28','09','93','escrita','06','11','93','parte','escrito','narrativa','de','experiência','pessoal','e','lisandra', 'um',
'experiência', 'um', 'narrativa', 'teu', 'de', 'experiência']

In [ ]:
# Transformar as listas em dicionário

dev = dict(zip(recorte_riogrande, lematizacaomodelo))

In [ ]:
# Exportar como o arquivo dev em json

with open("dev.json", 'w', encoding='utf-8') as json_file:
  json.dump(dev, json_file, ensure_ascii=False, indent=4)

## Treinamento

In [ ]:
!spacy init config -p trainable_lemmatizer config.cfg --lang pt --pipeline lemmatizer --force

⚠ To generate a more effective transformer-based config (GPU-only),
install the spacy-transformers package and re-run this command. The config
generated now does not use transformers.
ℹ Generated config template specific for your use case
- Language: pt
- Pipeline: lemmatizer
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
!pip install spacy-legacy

import srsly
from spacy.tokens import DocBin
from pathlib import Path
import warnings

def convert(lang: str, input_path: Path, output_path: Path):
    nlp = spacy.blank(lang)
    db = DocBin()
    data = srsly.read_json(input_path) # Lê os dados em arquivos jsons

    # Mudei a estrutura para lidar com diferentes estruturas de texto
    # Não alterar
    if isinstance(data, dict) and "text" in data and "annotations" in data:  # Formata 'train.json'
        texts = data["text"]
        labels = data["annotations"]["label"]
        for text, label in zip(texts, labels):
            doc = nlp.make_doc(text)
            doc.user_data["lemmas"] = [label]  # Store lemma as user data
            db.add(doc)
    elif isinstance(data, dict):  # Format for 'dev.json'
        for text, label in data.items():
            doc = nlp.make_doc(text)
            doc.user_data["lemmas"] = [label]  # Store lemma as user data
            db.add(doc)
    else:
        raise ValueError("Unsupported data format in JSON file.")

    db.to_disk(output_path)


# Aplicação
lang = "pt"
input_path = Path("/content/drive/MyDrive/Corpus_DeG/treinamento_spaCy/train.json")
output_path = Path("/content/drive/MyDrive/Corpus_DeG/treinamento_spaCy/train.spacy")
convert(lang, input_path, output_path)

input_path = Path("/content/drive/MyDrive/Corpus_DeG/treinamento_spaCy/dev.json")
output_path = Path("/content/drive/MyDrive/Corpus_DeG/treinamento_spaCy/dev.spacy")
convert(lang, input_path, output_path)

In [ ]:
# Carregar arquivo base de config e atualizar informações

from spacy.util import load_config

config = load_config("config.cfg")
config["corpora"]["train"] = {"@readers": "spacy.Corpus.v1", "path": "/content/drive/MyDrive/Corpus_DeG/treinamento_spaCy/train.spacy"}
config["corpora"]["dev"] = {"@readers": "spacy.Corpus.v1", "path": "/content/drive/MyDrive/Corpus_DeG/treinamento_spaCy/dev.spacy"}

with open("config.cfg", "w") as f:
    f.write(config.to_str())

In [ ]:
!spacy init fill-config /content/drive/MyDrive/Corpus_DeG/treinamento_spaCy/config.cfg /content/drive/MyDrive/Corpus_DeG/treinamento_spaCy/config.cfg

✔ Auto-filled config with all values
✔ Saved config
/content/drive/MyDrive/Corpus_DeG/treinamento_spaCy/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
!spacy debug config /content/drive/MyDrive/Corpus_DeG/treinamento_spaCy/config.cfg


============================= Config validation =============================

===================== Config validation for [initialize] =====================

====================== Config validation for [training] ======================
✔ Config is valid


In [ ]:
# Executar o treinamento

!spacy train config.cfg --output ./output --paths.train ./data/train.json --paths.dev ./data/dev.json

# Mensagem de erro encontrada:
# ValueError: [E955] Can't find table(s) lemma_lookup for language 'pt' in spacy-lookups-data.
# Make sure you have the package installed or provide your own lookup tables if no default lookups are available for your language

ℹ Saving to output directory: output
ℹ Using CPU

=========================== Initializing pipeline ===========================
Traceback (most recent call last):
  File "/usr/local/bin/spacy", line 8, in <module>
    sys.exit(setup_cli())
  File "/usr/local/lib/python3.10/dist-packages/spacy/cli/_util.py", line 87, in setup_cli
    command(prog_name=COMMAND)
  File "/usr/local/lib/python3.10/dist-packages/click/core.py", line 1157, in __call__
    return self.main(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/typer/core.py", line 743, in main
    return _main(
  File "/usr/local/lib/python3.10/dist-packages/typer/core.py", line 198, in _main
    rv = self.invoke(ctx)
  File "/usr/local/lib/python3.10/dist-packages/click/core.py", line 1688, in invoke
    return _process_result(sub_ctx.command.invoke(sub_ctx))
  File "/usr/local/lib/python3.10/dist-packages/click/core.py", line 1434, in invoke
    return ctx.invoke(self.callback, **ctx.params)
  File "/usr/local/lib/

In [ ]:
!spacy evaluate ./output/ ./output/dev.json

ℹ Using CPU
Traceback (most recent call last):
  File "/usr/local/bin/spacy", line 8, in <module>
    sys.exit(setup_cli())
  File "/usr/local/lib/python3.10/dist-packages/spacy/cli/_util.py", line 87, in setup_cli
    command(prog_name=COMMAND)
  File "/usr/local/lib/python3.10/dist-packages/click/core.py", line 1157, in __call__
    return self.main(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/typer/core.py", line 743, in main
    return _main(
  File "/usr/local/lib/python3.10/dist-packages/typer/core.py", line 198, in _main
    rv = self.invoke(ctx)
  File "/usr/local/lib/python3.10/dist-packages/click/core.py", line 1688, in invoke
    return _process_result(sub_ctx.command.invoke(sub_ctx))
  File "/usr/local/lib/python3.10/dist-packages/click/core.py", line 1434, in invoke
    return ctx.invoke(self.callback, **ctx.params)
  File "/usr/local/lib/python3.10/dist-packages/click/core.py", line 783, in invoke
    return __callback(*args, **kwargs)
  File "/usr/loc

In [ ]:
# Carregar o modelo treinado
nlp = spacy.load('./output/model-best')

# Testar com uma nova frase
texto = "Os gatos estão correndo no jardim"
doc = nlp(texto)

# Exibir os lemas dos tokens
for token in doc:
    print(f"{token.palavras} -> {token.lemmas}")

OSError: [E050] Can't find model './output/model-best'. It doesn't seem to be a Python package or a valid path to a data directory.